In [1]:
import os.path
from functools import partial

import bw2calc as bc
import bw2data as bd
import bw2io as bi

In [2]:
# bd.projects

In [3]:
bd.projects.set_current("default")

In [4]:
# bd.databases

## import Stratospheric Ozone Depletion method
 > Stratospheric O3 concentration, DU  


In [5]:
method_category = "OzoneDepletion_StratosphericO3Concentration"

In [6]:
SIMAPRO_DIR = os.path.abspath(os.path.join("../../from_simapro"))

In [7]:
STORE_DATA_TO = os.path.join(
    os.path.abspath(os.path.join("../../src/aesa_pbs/data"))
    + f"\\aesa_{method_category}.xlsx"
)

In [8]:
si = bi.SimaProLCIACSVImporter(
    filepath=os.path.join(SIMAPRO_DIR + f"\PBsLCIAv072_{method_category}.csv")
)

Extracted 1 methods in 0.01 seconds


In [9]:
# si.data[0].keys()

In [10]:
for method in si.data:
    print(method["name"], method["unit"])

('PBs-LCIA (baseline)', 'Stratospheric ozone depletion') DU


In [11]:
si.data[0]["exchanges"][0]#.keys()

{'amount': 1.41e-10,
 'CAS number': '010024-97-2',
 'categories': ('Air', '(unspecified)'),
 'name': 'Dinitrogen monoxide',
 'unit': 'kg'}

In [12]:
# link_by_name_and_categories = partial(
#     bi.strategies.link_iterable_by_fields,
#     other=bd.Database(bd.config.biosphere),
#     kind="biosphere",
#     fields=("name", "categories"),
# )

In [13]:
si.apply_strategies()
si.statistics()

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: link_iterable_by_fields
Applying strategy: match_subcategories
Applied 10 strategies in 0.39 seconds
1 methods
77 cfs
2 unlinked cfs


(1, 77, 2)

In [14]:
# list(si.unlinked)

In [15]:
# from the 5 unlinked:
# 1 has amount 0, can be dropped - Carbon dioxide, in air
# 2 others "Carbon dioxide" and "Carbon monoxide" are not in biosphere3, BUT
# "Carbon dioxide, fossil" 
# and "Carbon monoxide, fossil" and "Carbon monoxide, non-fossil" (with same CFs)
# have been linked (see check_equivalent_linked)
[(exc["name"], exc["categories"]) for exc in list(si.unlinked)]

[('Ethane, 1,2-dibromotetrafluoro-, Halon 2402', ('air',)),
 ('Methane, dibromodifluoro-, Halon 1202', ('air',))]

In [16]:
def see_unlinked(imported_methods):
    unlinked_exc_names = [
        exc_unlinked["name"] for exc_unlinked in list(imported_methods.unlinked)
    ]
    for ix in range(len(imported_methods.data)):
        print(imported_methods.data[ix]["name"][1])
        for exc in imported_methods.data[ix]["exchanges"]:
            if exc["name"] in unlinked_exc_names:
                print("\t",
                    exc["name"],
                    exc["categories"],
                    exc["amount"],
                )
        print("\n")
        # if exc["amount"] != 0

In [17]:
def check_equivalent_linked(imported_methods, list_names):
    for ix in range(len(imported_methods.data)):
        print(imported_methods.data[ix]["name"][1])
        for exc in imported_methods.data[ix]["exchanges"]:
            for x in list_names:
                if x in exc["name"]:
                    print("\t",
                        exc["name"],
                        exc["categories"],
                        exc["amount"],
                    )
        print("\n")

In [18]:
def remove_duplicates_if_any(imported_methods):
    for ix in range(len(imported_methods.data)):
        current_exc = len(imported_methods.data[ix]["exchanges"])
        wo_duplicates_exc = len(
            {tuple(o["input"]) for o in imported_methods.data[ix]["exchanges"]}
        )
        print(
            imported_methods.data[ix]["name"][1], ": ", current_exc, wo_duplicates_exc,
        )
        if current_exc != wo_duplicates_exc:
            unique_exchanges = {
                tuple(o["input"]): o for o in imported_methods.data[ix]["exchanges"]
            }
            imported_methods.data[ix]["exchanges"] = list(unique_exchanges.values())
            print("Duplicates removed.\n")
        else:
            print("No duplicates.\n")

In [19]:
see_unlinked(si)

Stratospheric ozone depletion
	 Ethane, 1,2-dibromotetrafluoro-, Halon 2402 ('air',) 5.39e-08
	 Methane, dibromodifluoro-, Halon 1202 ('air',) 1.14e-08




In [20]:
# check_equivalent_linked(si, ["Halon"])

In [21]:
b3 = bd.Database("biosphere3")
sorted([flow["name"] for flow in b3 if "halon" in flow["name"].lower()])

['Chlorothalonil',
 'Methane, bromo-, Halon 1001',
 'Methane, bromo-, Halon 1001',
 'Methane, bromo-, Halon 1001',
 'Methane, bromo-, Halon 1001',
 'Methane, bromo-, Halon 1001',
 'Methane, bromochlorodifluoro-, Halon 1211',
 'Methane, bromochlorodifluoro-, Halon 1211',
 'Methane, bromochlorodifluoro-, Halon 1211',
 'Methane, bromochlorodifluoro-, Halon 1211',
 'Methane, bromochlorodifluoro-, Halon 1211',
 'Methane, bromotrifluoro-, Halon 1301',
 'Methane, bromotrifluoro-, Halon 1301',
 'Methane, bromotrifluoro-, Halon 1301',
 'Methane, bromotrifluoro-, Halon 1301',
 'Methane, bromotrifluoro-, Halon 1301']

In [22]:
si.drop_unlinked()
si.statistics()

Applying strategy: drop_unlinked_cfs
Applied 1 strategies in 0.00 seconds
1 methods
75 cfs
0 unlinked cfs


(1, 75, 0)

In [23]:
# check if there are duplicates (both numbers should be equal)
remove_duplicates_if_any(si)

Stratospheric ozone depletion :  75 75
No duplicates.



In [ ]:
# Next si.write_excel(method_category) will create an excel file in 
# ~\Local\pylca\Brightway3\default###\output\lcia-matching-{method_category}.xlsx

# Go to created excel and apply manual corrections
#TODO: manual corrections should not be manual

# Store corrected excel to STORE_DATA_TO